##### Packages

In [1]:
from gurobipy import *
import pandas as pd

##### Read Data

In [2]:
store_info = pd.read_excel('OR_hw01_data.xlsx', 'Store')
stores = range(len(store_info['Store ID']))
expected_daily_demand = store_info['Expected Daily Demand (unit)']
store_xcoord = store_info['x-coordinate (km)']
store_ycoord = store_info['y-coordinate (km)']

dc_info = pd.read_excel('OR_hw01_data.xlsx', 'DC')
dcs = range(len(dc_info['Location ID']))
maintenance_costs = dc_info['Maintenance Cost ($/unit)']
construction_costs = dc_info['Construction Cost ($)']
dc_xcoord = dc_info['x-coordinate (km)']
dc_ycoord = dc_info['y-coordinate (km)']
dc_maxscale = dc_info['Maximum Scale (unit)']

S = int # replenishment cost

##### Build Model

In [3]:
eg1 = Model("eg1")

#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-29


##### Objective Fucntion

In [4]:
eg1.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(maintenance_costs[j] * quicksum(r[i][j] for i in stores) for j in dcs) \
    + quicksum(quicksum(r[i][j] * (wx[i][j] + wy[i][j]) for i in stores) for j in dcs)\
    ,GRB.MINIMIZE
)

##### Constraints

In [5]:
eg1.addConstrs((quicksum(r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1.addConstrs((quicksum(r[i][j] for i in stores) <= dc_maxscale[j] for j in dcs), "dcCapacity")
eg1.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [6]:
eg1.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4110 rows, 3010 columns and 6000 nonzeros
Model fingerprint: 0x81f26ee3
Model has 2000 quadratic objective terms
Variable types: 0 continuous, 3010 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+03]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 6.064400e+13
Presolve removed 4000 rows and 2010 columns
Presolve time: 0.06s
Presolved: 110 rows, 1000 columns, 2000 nonzeros
Found heuristic solution: objective 1133506.0000
Variable types: 0 continuous, 1000 integer (0 binary)
Found heuristic solution: objective 1130230.0000

Root relaxation: objective 4.490400e+05, 115 iterations, 0.01 seconds (0